In [29]:
# which model to use?
output_dir = '/Users/jnaiman/Downloads/tmp/byt5_inline_cite_ref_large/' 
snapshot = 'checkpoint-87000' # set to None to take last

# where are historical ground truths stored?
historical_dataset_dir = '/Users/jnaiman/Dropbox/wwt_image_extraction/OCRPostCorrection/historical_docs/groundtruth/'

# which types of GT to use?
types = ['plain', 'author location']

In [33]:
from transformers import T5ForConditionalGeneration, ByT5Tokenizer, AutoTokenizer
from glob import glob
import pickle
import pandas as pd

from sys import path
path.append('../')

from utils import fix_ocr

In [7]:
# Load pretrained model and tokenizer
tokenizer = AutoTokenizer.from_pretrained(
    "google/byt5-small",
    cache_dir=output_dir, 
    max_length=4096
)

model = T5ForConditionalGeneration.from_pretrained(
    "google/byt5-small",
    cache_dir=output_dir,
)

# overwriting the default max_length of 20 
tokenizer.model_max_length=4096
model.config.max_length=4096

In [9]:
if snapshot == None:
    snapshots = glob(output_dir+'checkpoint*')
    order = []
    for s in snapshots:
        order.append(s.split('-')[-1])
    argsort = np.argsort(np.array(order).astype('int'))
    snapshot = np.array(snapshots)[argsort][-1]
else:
    snapshot = output_dir + snapshot

ckpoint = snapshot.split('-')[-1]

In [11]:
snapshot

'/Users/jnaiman/Downloads/tmp/byt5_inline_cite_ref_large/checkpoint-87000'

In [12]:
# replace with ours
model = T5ForConditionalGeneration.from_pretrained(snapshot)

Read in our data:

In [24]:
historical_gt = glob(historical_dataset_dir + '*pickle')
len(historical_gt)

215

In [30]:
filenames = []; pages = []; sents = []; 
source = []; target = []
types_here = []
for histgt in historical_gt:
    with open(histgt,'rb') as f:
        dir_test = pickle.load(f)  
        
    filenames.append(dir_test['filename'])
    pages.append(dir_test['page'])
    sents.append(dir_test['sent'])
    source.append(dir_test['source'])
    target.append(dir_test['target'])
    types_here.append(dir_test['type'])
    
df_historical_test_all = pd.DataFrame({'input_text':source, 
                                       'target_text':target,
                                   'filename':filenames, 'page':pages,
                                   'sent num':sents, 'type':types_here})

Subset for just the ones that we have prefered types for:

In [31]:
df_historical_test = df_historical_test_all.loc[df_historical_test_all['type'].isin(types)]

In [32]:
df_historical_test.head()

,input_text,target_text,filename,page,sent num,type
0,Menon (1961) obtained a detailed intensity dis...,\cite{Menon1961} obtained a detailed intensity...,1965ApJ___141_1331R,0,13,plain
2,classification is given in the seventh column ...,classification is given in the seventh column ...,1957ApJ___125__636W,6,0,plain
3,All catalogued nebulae showing on any of the p...,All catalogued nebulae showing on any of the p...,1917ApJ____46___24P,13,2,plain
4,"It is the close coincidence of the Mg triplet,...",It is the close coincidence of the $Mg$ triple...,1951ApJ___113__700K,0,17,plain
5,7 which shows the mean of the Ha profiles from...,7 which shows the mean of the H$\alpha$ profil...,1995AJ____109__350H,2,17,plain


Do inference:

In [42]:
imod = 10
skip_specials = True
verbose = True

outputs = []
for i in range(len(df_historical_test)):
    if i%imod == 0: print('on', i,'of',len(df_historical_test))
    text = df_historical_test.iloc[i]['input_text']
    inputs = tokenizer(text, padding="longest", return_tensors="pt")
    output = model.generate(**inputs)
    output_text = tokenizer.decode(output[0], skip_special_tokens=skip_specials, clean_up_tokenization_spaces=True)
    outputs.append(output_text)
    if verbose:
        print('i=', i)
        print('input text :', text)
        print('target text:', df_historical_test.iloc[i]['target_text'])
        print('output text:', output_text)
        print('')
    del inputs
    del output
    del output_text
    
df_historical_test['predicted_text'] = outputs

on 0 of 197


/Users/jnaiman/opt/anaconda3/envs/wwtTransformers/lib/python3.9/site-packages/transformers/generation/utils.py:1313: UserWarning: Using `max_length`'s default (4096) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


i= 0
input text : Menon (1961) obtained a detailed intensity distribution across the nebula at 3.75-cm wavelength.
target text: \cite{Menon1961} obtained a detailed intensity distribution across the nebula at 3.75-cm wavelength.
output text: Menon (1961) obtained a detailed intensity distribution across the nebula at 3.75-cm wavelength.

i= 1
input text : classification is given in the seventh column for consistency, since the largest number of classifications available are those made by him.
target text: classification is given in the seventh column for consistency, since the largest number of classifications available are those made by him.
output text: classification is given in the seventh column for consistency, since the largest number of classifications available are those made by him.

i= 2
input text : All catalogued nebulae showing on any of the photographs here discussed have been inserted in the list; but those which appear only incidentally are described in the text under 

In [45]:
df_historical_test

,input_text,target_text,filename,page,sent num,type,predicted_text
0,Menon (1961) obtained a detailed intensity dis...,\cite{Menon1961} obtained a detailed intensity...,1965ApJ___141_1331R,0,13,plain,Menon (1961) obtained a detailed intensity dis...
2,classification is given in the seventh column ...,classification is given in the seventh column ...,1957ApJ___125__636W,6,0,plain,classification is given in the seventh column ...
3,All catalogued nebulae showing on any of the p...,All catalogued nebulae showing on any of the p...,1917ApJ____46___24P,13,2,plain,All catalogued nebulae showing on any of the p...
4,"It is the close coincidence of the Mg triplet,...",It is the close coincidence of the $Mg$ triple...,1951ApJ___113__700K,0,17,plain,"It is the close coincidence of the Mg triplet,..."
5,7 which shows the mean of the Ha profiles from...,7 which shows the mean of the H$\alpha$ profil...,1995AJ____109__350H,2,17,plain,\ref{fig:ha_profiles} which shows the mean of ...
...,...,...,...,...,...,...,...
210,"However, this model will not operate in sys-te...","However, this model will not operate in sys-te...",1983ApJ___275__549J,7,7,plain,"However, this model will not operate in sys-t..."
211,"N r1] is stronger than He, which is typical of...",[N \MakeUppercase{\romannumeral 2}] is stronge...,1974ApJ___191__645R,1,1,plain,"N ] is stronger than He, which is typical of t..."
212,For all clusters the average V and B — V and t...,For all clusters the average $V$ and $B - V$ a...,1975ApJ___196__369F,8,11,plain,For all clusters the average $V$ and $B$ $-$ $...
213,"tektites we have measured or, if tektite mater...","tektites we have measured or, if tektite mater...",1961ApJ___133__706O,1,0,plain,"tektites we have measured or, if tektite mater..."
